In [1]:
!pip install moviepy
!pip install ultralytics
# Core Libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from scipy import signal
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from tqdm import tqdm

# YOLO for object detection
from ultralytics import YOLO

# Fix deprecated np aliases (for older code)
np.int = int
np.bool = bool
np.float = float


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
# Lane detection & thresholds
LANE_SETTINGS = (50, 150, 5)  # canny_low, canny_high, blur
THRESHOLDS = (0.7, 0.4)      # high, low confidence

# Object detection sets & priorities
VEHICLES = {'car', 'truck', 'bus'}
PEDESTRIANS = {'person'}
PRIORITY = {'person': 1, 'car': 3, 'truck': 4}
COLORS = {'person': (255, 0, 0), 'vehicle': (0, 255, 255)}

# Lists to store history
confidence_list = []
detection_list = []
decision_list = []


In [3]:
# ============================================================================
# SIMPLE LANE DETECTOR
# ============================================================================

def detect_lanes(image):
    """
    Simple lane detection
    Uses: TUPLES (config), NUMPY (arrays), LISTS (storage)
    """
    # Unpack tuple
    canny_low, canny_high, blur = LANE_SETTINGS

    # NUMPY: Image processing
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur_img = cv2.GaussianBlur(gray, (blur, blur), 0)
    edges = cv2.Canny(blur_img, canny_low, canny_high)

    # Simple ROI mask
    height, width = image.shape[:2]
    mask = np.zeros_like(edges)

    # NUMPY: Array for polygon
    polygon = np.array([[
        (width * 0.1, height),
        (width * 0.45, height * 0.6),
        (width * 0.55, height * 0.6),
        (width * 0.9, height)
    ]], dtype=np.int32)

    cv2.fillPoly(mask, polygon, 255)
    masked = cv2.bitwise_and(edges, mask)

    # Detect lines
    lines = cv2.HoughLinesP(masked, 2, np.pi/180, 50, minLineLength=40, maxLineGap=100)

    # Draw lines
    result = image.copy()
    line_count = 0

    if lines is not None:
        line_count = len(lines)
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(result, (x1, y1), (x2, y2), (0, 255, 0), 5)

    # Calculate confidence using NUMPY
    confidence = np.clip(line_count / 10.0, 0, 1)

    # TUPLE return
    return (result, confidence)



In [4]:
# SIMPLE OBJECT DETECTOR
def detect_objects(image, model):
    """
    Simple object detection
    Uses: SETS (filtering), DICT (data), LISTS (collection)
    """
    results = model(image, conf=0.5, verbose=False)

    # LIST to store detections
    detections = []
    result_img = image.copy()

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = float(box.conf[0])
            cls = model.names[int(box.cls[0])]

            # SET membership test (O(1))
            if cls in VEHICLES or cls in PEDESTRIANS:

                # DICT to store detection info
                det_dict = {
                    'class': cls,
                    'confidence': conf,
                    'box': (x1, y1, x2, y2),  # TUPLE
                    'center': ((x1+x2)//2, (y1+y2)//2),  # TUPLE
                    'priority': PRIORITY.get(cls, 5)
                }

                detections.append(det_dict)

                # Draw box
                color = COLORS['person'] if cls in PEDESTRIANS else COLORS['vehicle']
                cv2.rectangle(result_img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(result_img, f"{cls} {conf:.2f}", (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # TUPLE return
    return (result_img, detections)


In [5]:
# SIMPLE DECISION ENGINE
def make_decision(detections, confidence):
    """
    Simple decision logic
    Uses: LISTS (filtering), TUPLES (thresholds), DICT (access)
    """
    high_conf, low_conf = THRESHOLDS  # Unpack tuple

    # Check confidence
    if confidence < low_conf:
        return ("SLOW DOWN", "Low lane confidence")

    # LIST COMPREHENSION: Filter pedestrians
    pedestrians = [d for d in detections if d['class'] in PEDESTRIANS]

    if pedestrians:
        return ("STOP", "Pedestrian detected")

    # LIST COMPREHENSION: Filter vehicles
    vehicles = [d for d in detections if d['class'] in VEHICLES]

    if vehicles:
        # Sort by priority (using dict value)
        vehicles.sort(key=lambda x: x['priority'])
        return ("CAUTION", f"{vehicles[0]['class']} ahead")

    return ("CONTINUE", "Path clear")

In [6]:
# ============================================================================
# PANDAS LOGGER
# ============================================================================

def create_log_dataframe(detection_list):
    """
    Create Pandas DataFrame from list of dicts
    Uses: PANDAS (DataFrame, operations)
    """
    # LIST of DICTS to DataFrame
    df = pd.DataFrame(detection_list)

    if df.empty:
        return df

    # PANDAS operations
    print("\n[PANDAS] Detection Statistics:")
    print(f"  Total detections: {len(df)}")
    print(f"  By class:\n{df['class'].value_counts()}")
    print(f"  Average confidence: {df['confidence'].mean():.3f}")

    return df


In [7]:
# SCIPY SMOOTHING

def smooth_confidence(confidence_list):
    """
    Smooth confidence using SciPy
    Uses: SCIPY (signal processing), NUMPY (arrays)
    """
    if len(confidence_list) < 5:
        return confidence_list

    # LIST to NUMPY array
    conf_array = np.array(confidence_list)

    # SCIPY: Savitzky-Golay filter
    window = min(11, len(conf_array))
    if window % 2 == 0:
        window -= 1

    smoothed = signal.savgol_filter(conf_array, window, 2)

    return smoothed


In [8]:
# ============================================================================
# MATPLOTLIB VISUALIZATION
# ============================================================================

def create_simple_dashboard(confidence_list, detection_list, decision_list):
    """
    Simple 2x2 dashboard
    Uses: MATPLOTLIB (subplots, various plots)
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('ADAS Analysis Dashboard', fontsize=14, fontweight='bold')

    # Plot 1: Confidence over time
    if confidence_list:
        conf_array = np.array(confidence_list)
        frames = np.arange(len(conf_array))

        axes[0, 0].plot(frames, conf_array, 'b-', alpha=0.5, label='Raw')

        # Add smoothed line using SciPy
        if len(conf_array) >= 5:
            smoothed = smooth_confidence(confidence_list)
            axes[0, 0].plot(frames, smoothed, 'r-', linewidth=2, label='Smoothed')

        axes[0, 0].set_title('Lane Confidence Trend')
        axes[0, 0].set_xlabel('Frame')
        axes[0, 0].set_ylabel('Confidence')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: Decision distribution
    if decision_list:
        # COUNT decisions using DICT
        decision_counts = {}
        for cmd, _ in decision_list:
            decision_counts[cmd] = decision_counts.get(cmd, 0) + 1

        commands = list(decision_counts.keys())
        counts = list(decision_counts.values())

        axes[0, 1].bar(commands, counts, color='steelblue')
        axes[0, 1].set_title('Decision Distribution')
        axes[0, 1].set_ylabel('Count')
        axes[0, 1].tick_params(axis='x', rotation=45)

    # Plot 3: Confidence histogram
    if confidence_list:
        conf_array = np.array(confidence_list)
        axes[1, 0].hist(conf_array, bins=15, color='green', alpha=0.7, edgecolor='black')

        # NUMPY: Add mean line
        mean = np.mean(conf_array)
        axes[1, 0].axvline(mean, color='red', linestyle='--',
                          label=f'Mean: {mean:.2f}')

        axes[1, 0].set_title('Confidence Distribution')
        axes[1, 0].set_xlabel('Confidence')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].legend()

    # Plot 4: Detection counts by class
    if detection_list:
        # Create PANDAS DataFrame
        df = pd.DataFrame(detection_list)
        class_counts = df['class'].value_counts()

        class_counts.plot(kind='barh', ax=axes[1, 1], color='coral')
        axes[1, 1].set_title('Detections by Class')
        axes[1, 1].set_xlabel('Count')

    plt.tight_layout()
    plt.savefig('dashboard.png', dpi=150, bbox_inches='tight')
    print("\n✓ Dashboard saved: dashboard.png")
    plt.close()

In [9]:
# ============================================================================
# LEVEL 1: LANE ONLY
# ============================================================================

def level_1_lane_only(image_path):
    """Level 1: Just lane detection"""
    print("\n" + "="*60)
    print("LEVEL 1: LANE DETECTION")
    print("="*60)

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    result, confidence = detect_lanes(image_rgb)

    # Add text
    cv2.putText(result, f"Confidence: {confidence:.2f}", (20, 40),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(image_rgb)
    axes[0].set_title("Original")
    axes[0].axis('off')

    axes[1].imshow(result)
    axes[1].set_title(f"Lane Detection (Conf: {confidence:.2f})")
    axes[1].axis('off')

    plt.tight_layout()
    plt.savefig('level1_output.png', dpi=150)
    print("✓ Saved: level1_output.png")

In [10]:
# ============================================================================
# LEVEL 2: OBJECTS ONLY
# ============================================================================

def level_2_objects_only(image_path):
    """Level 2: Just object detection"""
    print("\n" + "="*60)
    print("LEVEL 2: OBJECT DETECTION")
    print("="*60)

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    model = YOLO('yolov8n.pt')
    result, detections = detect_objects(image_rgb, model)

    # Display
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(image_rgb)
    axes[0].set_title("Original")
    axes[0].axis('off')

    axes[1].imshow(result)
    axes[1].set_title(f"Objects Detected: {len(detections)}")
    axes[1].axis('off')

    plt.tight_layout()
    plt.savefig('level2_output.png', dpi=150)
    print(f"✓ Detected {len(detections)} objects")
    print("✓ Saved: level2_output.png")

In [11]:
# ============================================================================
# LEVEL 3: COMPLETE ON IMAGE
# ============================================================================

def level_3_complete_image(image_path):
    """Level 3: Complete system on single image"""
    print("\n" + "="*60)
    print("LEVEL 3: COMPLETE SYSTEM")
    print("="*60)

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    model = YOLO('yolov8n.pt')

    # Step 1: Lanes
    lane_img, confidence = detect_lanes(image_rgb)

    # Step 2: Objects
    detected_img, detections = detect_objects(lane_img, model)

    # Step 3: Decision
    command, reason = make_decision(detections, confidence)

    # Add overlay
    final = detected_img.copy()
    cv2.rectangle(final, (0, 0), (final.shape[1], 80), (0, 0, 0), -1)
    final = cv2.addWeighted(detected_img, 0.7, final, 0.3, 0)

    color = (255, 0, 0) if "STOP" in command else (0, 255, 0)
    cv2.putText(final, command, (20, 35),
               cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.putText(final, reason, (20, 65),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Display
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(image_rgb)
    axes[0].set_title("Original")
    axes[0].axis('off')

    axes[1].imshow(final)
    axes[1].set_title("Complete ADAS")
    axes[1].axis('off')

    plt.tight_layout()
    plt.savefig('level3_output.png', dpi=150)
    print(f"✓ Command: {command}")
    print(f"✓ Reason: {reason}")
    print("✓ Saved: level3_output.png")

In [12]:
# ============================================================================
# LEVEL 4: VIDEO PROCESSING
# ============================================================================

def level_4_video(video_path, output_path='output.mp4'):
    """Level 4: Process entire video"""
    print("\n" + "="*60)
    print("LEVEL 4: VIDEO PROCESSING")
    print("="*60)

    # Open video
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video: {width}x{height}, {fps} FPS, {total} frames")

    # Output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Load model
    model = YOLO('yolov8n.pt')

    frame_num = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_num += 1
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process
        lane_img, confidence = detect_lanes(frame_rgb)
        detected_img, detections = detect_objects(lane_img, model)
        command, reason = make_decision(detections, confidence)

        # Store in LISTS
        confidence_list.append(confidence)
        detection_list.extend(detections)
        decision_list.append((command, reason))

        # Add overlay
        final = detected_img.copy()
        cv2.rectangle(final, (0, 0), (width, 90), (0, 0, 0), -1)
        final = cv2.addWeighted(detected_img, 0.7, final, 0.3, 0)

        color = (255, 0, 0) if "STOP" in command else (0, 255, 0)
        cv2.putText(final, command, (20, 35),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
        cv2.putText(final, reason, (20, 65),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(final, f"Frame {frame_num}/{total}", (width-200, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Write
        final_bgr = cv2.cvtColor(final, cv2.COLOR_RGB2BGR)
        out.write(final_bgr)

        if frame_num % 30 == 0:
            print(f"  Progress: {frame_num}/{total} frames")

    cap.release()
    out.release()

    print(f"\n✓ Video saved: {output_path}")
    print(f"✓ Processed {frame_num} frames")

    # Create Pandas log
    df = create_log_dataframe(detection_list)
    if not df.empty:
        df.to_csv('detection_log.csv', index=False)
        print("✓ CSV saved: detection_log.csv")

    # Create dashboard
    create_simple_dashboard(confidence_list, detection_list, decision_list)

    # Print summary
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)

    # NUMPY stats
    if confidence_list:
        conf_array = np.array(confidence_list)
        print(f"\n[NumPy] Confidence:")
        print(f"  Mean: {np.mean(conf_array):.3f}")
        print(f"  Std: {np.std(conf_array):.3f}")

    # LIST stats
    print(f"\n[Lists] Data collected:")
    print(f"  Confidence points: {len(confidence_list)}")
    print(f"  Total detections: {len(detection_list)}")
    print(f"  Decisions made: {len(decision_list)}")

    # DICT stats
    decision_counts = {}
    for cmd, _ in decision_list:
        decision_counts[cmd] = decision_counts.get(cmd, 0) + 1

    print(f"\n[Dict] Decision counts:")
    for cmd, count in decision_counts.items():
        print(f"  {cmd}: {count}")



In [13]:
from google.colab import files

if __name__ == "__main__":
    print("\n" + "="*70)
    print("🚗 SIMPLE ADAS SYSTEM - ACADEMIC PROJECT")
    print("="*70)
    print("""
Select one of the following options:

    Level 1 - Lane detection only (image)
    Level 2 - Object detection only (image)
    Level 3 - Complete system (image)
   Level 4 - Full video processing demo
    """)

    try:
        choice = int(input("Enter your choice (1-4): "))
    except ValueError:
        print("Invalid input. Please enter a number between 1 and 4.")
        exit()

    # Handle image/video upload
    if choice in [1, 2, 3]:
        print("\nPlease upload your IMAGE file (e.g., road.jpg):")
        uploaded = files.upload()
        image_path = list(uploaded.keys())[0]
    elif choice == 4:
        print("\n Please upload your VIDEO file (e.g., challenge.mp4):")
        uploaded = files.upload()
        video_path = list(uploaded.keys())[0]
    else:
        print("Invalid choice. Please select between 1 and 4.")
        exit()

    print("\nProcessing...")

    # Run selected level
    if choice == 1:
        level_1_lane_only(image_path)
    elif choice == 2:
        level_2_objects_only(image_path)
    elif choice == 3:
        level_3_complete_image(image_path)
    elif choice == 4:
        output_video = "adas_output.mp4"
        level_4_video(video_path, output_video)
        print(f"🎬 Video saved as: {output_video}")

    print("="*70)



🚗 SIMPLE ADAS SYSTEM - ACADEMIC PROJECT

Select one of the following options:

    Level 1 - Lane detection only (image)
    Level 2 - Object detection only (image)
    Level 3 - Complete system (image)
   Level 4 - Full video processing demo
    
Enter your choice (1-4): 4

 Please upload your VIDEO file (e.g., challenge.mp4):


Saving project_video.mp4 to project_video.mp4

Processing...

LEVEL 4: VIDEO PROCESSING
Video: 1280x720, 25 FPS, 1260 frames
  Progress: 30/1260 frames
  Progress: 60/1260 frames
  Progress: 90/1260 frames
  Progress: 120/1260 frames
  Progress: 150/1260 frames
  Progress: 180/1260 frames
  Progress: 210/1260 frames
  Progress: 240/1260 frames
  Progress: 270/1260 frames
  Progress: 300/1260 frames
  Progress: 330/1260 frames
  Progress: 360/1260 frames
  Progress: 390/1260 frames
  Progress: 420/1260 frames
  Progress: 450/1260 frames
  Progress: 480/1260 frames
  Progress: 510/1260 frames
  Progress: 540/1260 frames
  Progress: 570/1260 frames
  Progress: 600/1260 frames
  Progress: 630/1260 frames
  Progress: 660/1260 frames
  Progress: 690/1260 frames
  Progress: 720/1260 frames
  Progress: 750/1260 frames
  Progress: 780/1260 frames
  Progress: 810/1260 frames
  Progress: 840/1260 frames
  Progress: 870/1260 frames
  Progress: 900/1260 frames
  Progress: 930/1260 frames
  Progress